In [19]:
from pathlib import Path
from torch.utils.data import DataLoader
import torch
import pandas as pd
from pvimage import features
import imageio.v3 as iio
import numpy as np
from PIL import Image

from utils import train_functions, viz_functions, img_functions

In [2]:
category_mapping = {0: "dark", 1: "busbar", 2: "crack", 3: "non-cell"}
root = Path(
    "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Channeled_Combined_CWRU_LBNL_ASU_No_Empty/"
)
img_root = root / "img" / "all"
weight_path = "/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Channeled_Combined_CWRU_LBNL_ASU_No_Empty/checkpoints/Channeled_Combined_CWRU_LBNL_ASU_No_Empty10/epoch_19/model.pt"


train_dataset, val_dataset = train_functions.load_dataset(root)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
device, model = train_functions.load_device_and_model(category_mapping, existing_weight_path=weight_path)

In [3]:
idx = 0
threshold = 0.5

# viz_functions.channeled_inference_and_show(
#         val_loader, device, model, category_mapping, idx
#     )

In [4]:
# Images
img, mask = val_loader.dataset.__getitem__(idx)
img = img.to(device)
raw_img, _ = val_loader.dataset.__getraw__(idx)

# Mask
gt_mask = mask.cpu().numpy()

# Inference
logits = model(img.unsqueeze(0)).detach().cpu()
probs = torch.sigmoid(logits)
pred_mask = (probs > threshold).float().squeeze(0).numpy()

In [5]:
img_path = val_loader.dataset.__get_img_path__(idx)


In [6]:
print(img_path)

/Users/ojas/Desktop/saj/SANDIA/pvcracks_data/Channeled_Combined_CWRU_LBNL_ASU_No_Empty/img/val/mxy_sa19965_sub_EL_9-c02.tiff


In [22]:
paths = []

array_new = []
for i in range(585):
    img_path = val_loader.dataset.__get_img_path__(idx)
    dat = iio.imread(img_path)
    # dat = Image.open(img_path)
    datmean = dat.mean()
    dat = (dat > datmean).astype(np.float32)
    array_new.append(dat)
    paths.append(img_path)

array_new = np.stack(array_new)
print("Size of new array with cracked images %s" % str(array_new.shape))

testimg = array_new[-99:, :, :, 1:2]
testpaths = paths[-99:]
dfinfo = pd.DataFrame()
dfinfo["impath"] = testpaths

dffeaturesreal = features.feature_extraction_crack_mask(testimg, dfinfo)

dffeaturesreal.head()

Size of new array with cracked images (585, 400, 400, 3)


,cell_number,i,prop.perimeter,slope,prop.convex_area,prop.area,prop.orientation
0,mxy_sa19965_sub_EL_9,16,637.556349,3.467274,3826.0,3444.0,1.566084
0,mxy_sa19965_sub_EL_9,12,376.970563,-0.166905,2104.0,2000.0,-1.570107
0,mxy_sa19965_sub_EL_9,3,261.313708,-0.281977,1431.0,1366.0,-1.568243
0,mxy_sa19965_sub_EL_9,15,104.828427,-0.223593,499.0,483.0,-1.554585
0,mxy_sa19965_sub_EL_9,7,104.828427,-0.223593,499.0,483.0,-1.554585


In [26]:
image_tensors = []
masks = []

mask_index = -99
# GO BETWEEEN PATH / INDEX /?

preds = []

for p in testpaths:
    img, _ = val_loader.dataset.__getitem__(mask_index)
    img = img.to(device)
    # _, mask = val_loader.dataset.__getraw__(idx)
    # get associated mask path (sometihng -99 onwards ??)
    # mask = np.load(mask)
    
    # transformers = img_functions.Compose(
    #     [
    #         img_functions.ChanneledFixResize(256),
    #         img_functions.ToTensor(),
    #         img_functions.Normalize(),
    #     ]
    # )
    # img_tensor = transformers(image, ).to(device)
    # image_tensors.append(img.unsqueeze(0))
    # masks.append(mask)
    
    logits = model(img.unsqueeze(0)).detach().cpu()
    probs = torch.sigmoid(logits)
    pred_mask = (probs > threshold).float().squeeze(0).numpy()
    preds.append(pred_mask)
    
    idx += 1
    
print(preds)

# testtensors = torch.stack(image_tensors)
# logits = model(testtensors).detach().cpu()
# probs = torch.sigmoid(logits)
# pred_mask = (probs > threshold).float().squeeze(0).numpy()
testimg_reshaped = np.reshape(preds, (99, 400, 400, 1))

# testpreds = torch.stack(preds)

dffeaturesout = features.feature_extraction_crack_mask(testimg_reshaped, dfinfo)

dffeaturesout.head()

[array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]],
   

ValueError: cannot reshape array of size 25952256 into shape (99,400,400,1)

realizing some problem here

the vae example is comparing an imag and a (reconstructed) image

here i would want to compare masks and masks -- both 3d arrays (4-channeled 2d arrays)

but I'm not sure if pvimage actually enables that nor how it's supposed to work
